In [2]:
import pandas as pd
import datacube
import geopandas as gpd

dc = datacube.Datacube()

product = 'wofs'
date_from = '2025-01-01'
date_to = '2025-12-31'
output = 'wofs_2025.geojson'

f = open('landsat_scenes.txt')
a = f.readlines()
scenes = []
for i in a:
    scenes.append(i.replace('"','').replace('\n','').replace(',', ''))

def do_counts(product, scenes, date_from, date_to):
    data = {
        "platform": [],
        "scene_id": [],
        "count": [],
    }
    df = pd.DataFrame(data)
    if (product == 'wofs'):
        derivative_product = 'ga_ls_wo_3'
    elif (product == 'fc'):
        derivative_product = 'ga_ls_fc_3'
    for scene in scenes:
        # perform level 2 count
        product_count = len(dc.find_datasets(product=derivative_product, dataset_maturity="final", region_code=scene, time=(date_from, date_to)))
        label = ''
        data = {
            "product": [product],
            "region_code": [scene],
            "count": [product_count],
        }
        df_new_rows = pd.DataFrame(data)
        df = pd.concat([df, df_new_rows])
    return df

footprints_fpath = 'landsat_wrs2_descending.geojsonl'
df_ls_wrs2 = gpd.read_file(footprints_fpath)

df = do_counts(product, scenes, date_from, date_to)

merged_df = df_ls_wrs2.merge(df, on="region_code", how="left")

clean_df = merged_df.dropna(thresh=4)

legend_fpath = 'legend.geojson'
df_legend = gpd.read_file(legend_fpath)

merged_gdf = pd.concat([df_legend, clean_df], ignore_index=True)

merged_gdf.to_file(output, driver="GeoJSON")